In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import string
import sys
import re
import emoji
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, sent_tokenize, MWETokenizer
from pyvi import ViTokenizer
from unicodedata import normalize
pd.set_option('max_colwidth', -1)
from hdfs import InsecureClient
import glob

/home/polynote/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  from ipykernel import kernelapp as app


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import pickle
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
path_data = '/home/polynote/notebooks/ngaptq/fb_keyword/read_preprocess_data/'
all_files = glob.glob(path_data + "/fb_july.csv")

In [4]:
input_data = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None,sep="|")
    input_data.append(df)
input_data = pd.concat(input_data, axis=0, ignore_index=True)

In [5]:
input_data = input_data[input_data['clean_content'].notnull()]

Load keyword vocab và model

In [6]:
path_file = '/home/polynote/notebooks/ngaptq/fb_keyword/read_preprocess_data/train_model'
path_keyword = path_file + "/keyword_label.txt"
path_tfidf = path_file + "/tfidf_train.pkl"
path_model = path_file + "/model_train.pkl"
path_keyword_vec = path_file + "/vectorize_keyword_train.pkl"

In [7]:
load_tfidf = pickle.load(open(path_tfidf,'rb'))
load_model = pickle.load(open(path_model,'rb'))
load_keyword_vectorizer = pickle.load(open(path_keyword_vec,'rb'))
df_keyword = pd.read_csv(path_keyword,sep = '|',error_bad_lines=False,encoding='utf-8')

In [8]:
labels = df_keyword['label'].tolist() # list các label
keywords = df_keyword['clean_key'].apply(lambda x: x.split(",")).tolist()  # keyword dùng để gán nhãn
thresholds = df_keyword['threshold_keyword'].to_list() # threshold số từ xuất hiện để dán nhãn
except_keywords = df_keyword['keyword_except'].apply(lambda x: x.split(",")).to_list() # các keyword loại trừ
thresholds_proba = df_keyword['threshold_proba'].to_list() # ngưỡng xác suất để gán nhãn theo model cho từng nhãn

In [9]:
# ''' Tạo ma trận các từ có keyword gán nhãn và keyword ngoại lệ'''
# vocabulary = ','.join(df_keyword['clean_key']) + ','.join(df_keyword['keyword_except']) 
# vocabulary = list({vocabulary}) 
# vectorizer = CountVectorizer(binary=True, ngram_range=(1,5))
# vectorizer.fit(vocabulary)

Predict data

In [10]:
# Xóa các items có values bằng 0
def remove_dict(a):
    a = {k: v for k, v in a.items() if v > 0}
    return a

In [11]:
# sắp xếp dict theo thứ tự values giảm dần, và chỉ lấy 5 items đầu tiên
def order_dict(a):
    a = dict(sorted(a.items(), key=lambda t: t[1], reverse=True)[:5])
    return a

In [15]:
'''gán label cho input'''
def predict_label(input_data,labels, keywords,thresholds,except_keywords,thresholds_proba):
    '''
    Gán nhãn cho bài viết : bài viết xuất hiện số từ trong tập keyword > threshold và không xuất hiện
    từ trong nhóm keyword_except
    '''
    # Dataframe theo keyword
    transform_data = load_keyword_vectorizer.transform(input_data['clean_content'])
    label_keyword = pd.DataFrame.sparse.from_spmatrix(transform_data)
    label_keyword.columns = load_keyword_vectorizer.get_feature_names()
    # Dataframe theo model ML
    label_model = pd.DataFrame(load_model.predict_proba(load_tfidf.transform(input_data['clean_content'])))
    label_model.columns = labels
    for label, keyword,threshold,except_keyword,threshold_proba in zip(labels, keywords,thresholds,except_keywords,thresholds_proba):
        '''Tạo loop để trả kết quả labeling cho từng nhãn'''
        label_keyword['keyword'] = label_keyword.loc[:,label_keyword.columns.intersection(keyword)].sum(axis=1)
        label_keyword['keyword_except'] = label_keyword.loc[:,label_keyword.columns.intersection(except_keyword)].sum(axis=1)
        label_keyword['sum_keyword'] = label_keyword['keyword'] - 1000*label_keyword['keyword_except']
        label_keyword[label] = label_keyword['sum_keyword'].apply(lambda x:1 if x >= threshold else 0) # số lượng keyword xuất hiện > threshold 
        label_result = label_keyword.loc[:,label_keyword.columns.intersection(label)]
        label_model[label] = label_model[label] - 1000*label_keyword['keyword_except']
        label_model[label] = label_model[label].apply(lambda x:x if x >= threshold_proba else 0) # chỉ giữ lại xác suất nếu xác suất model > ngưỡng xác xuất của nhãn 
    label_result['dict_keyword'] = label_result.to_dict(orient='records') #tạo dict cho label theo keyword
    label_result['dict_keyword'] = label_result['dict_keyword'].apply(lambda x: remove_dict(x))
    label_result['dict_model'] = label_model.to_dict(orient='records')
    label_result['dict_model'] = label_result['dict_model'].apply(lambda x: remove_dict(x))
    label_result.apply(lambda x: x['dict_model'].update(x['dict_keyword']),axis=1) # merge 2 dict lại với nhau
    label_result['dict_final'] = label_result['dict_model'].apply(lambda x: order_dict(x)) # chỉ lấy tối đa 5 nhãn có xác suất lớn nhất
    df_label = label_result['dict_final'] 
    input_data = input_data.merge(df_label,left_index=True, right_index=True ) 
    return input_data.loc[:,['post_id','clean_content','dict_final']]

In [18]:
%%time
result = predict_label(input_data,labels, keywords,thresholds,except_keywords,thresholds_proba)

CPU times: user 8min 9s, sys: 24.5 s, total: 8min 34s
Wall time: 8min 4s


In [19]:
result.tail(50)

,post_id,clean_content,dict_final
325628,1.655136e+15,đầu tháng cuối tuần muốn mua gì cứ ib em nhé không ngại nắng không ngại mưa không tốn tiền qua nhật cũng mua được hàng chất_lượng cam_kết hàng chính hãng phát_hiện hàng giả đền_bù numbertoken lần để lớp trang_điểm được tự_nhiên và lâu bị trôi hơn trong mùa hè nắng_nóng các nàng đừng bỏ_qua những món đồ trang_điểm nh ẹ nhàng tự_nhiên đến từ xứ_sở hoa anh_đào dưới đây nhé nhilê cam_kết bán hàng chuẩn nội_địa nhật tất_cả các sản_phẩm đều có nguồn_gốc rõ_ràng từ store amazon jp rakuten jp hotline,{}
325629,4.288895e+15,đầu_tư vào đồng_tiền kỹ_thuật_số tiền mã_hoá không còn là một cơ_hội mơ_hồ như trong suy_nghĩ của nhiều người năm numbertoken đồng_tiền kỹ_thuật_số đầu_tiên trên thế_giới ra_đời với tên gọi là một đồng_tiền mang tính_cách mạng xuất_hiện trên internet mở ra một giải_pháp thanh_toán hoàn_hảo trên phạm_vi toàn_cầu với khả_năng có_thể được gửi trực_tiếp đến bất_cứ nơi nào trên thế_giới ngay_lập_tức với chi_phí gần như bằng numbertoken chỉ trong vòng numbertoken năm trị_giá đồng bitcoin tăng từ numbertoken numbertoken lên numbertoken numbertoken tăng gấp hơn numbertoken nghìn lần giá_trị của đồng vượt xa tất_cả các đồng_tiền usd euro v v và trở_thành đồng_tiền thành_công nhất trong lịch_sử tiền_tệ thế_giới nhưng tính đến nay đồng đã khai_thác quá lâu_dài giá_trị của nó củng đã lớn tiềm_năng tăng_trưởng giá_trị không còn nhiều số_lượng khai_thác củng không còn nhiều và dĩ_nhiên cơ_hội dành cho chúng_ta sở_hữu nó củng không còn nhiều nữa do_vậy nếu đã từng bỏ lỡ_cơ hội đầu_tư vào thì đồng_tiền mã_hoá chính là cơ_hội lịch_sử lặp lại đáng_giá hàng tỷ đồng dành cho tất_cả chúng_ta ilcoin là một loại tiền điện_tử được xây_dựng trên một hệ_thống blockchain độc_đáo và do_đó sở_hữu những phần công_nghệ nổi_bật như cp và giao_thức rift ilcoin đầu_tiên được khai_thác vào tháng numbertoken năm numbertoken khi ilcoin được ra_mắt lần đầu_tiên nó chỉ là một hệ_thống sha numbertoken proof of work pow tức_là một bản_sao btc đã có một_số phát_triển trong những năm qua có_thể được nhìn thấy trong lộ_trình ngày_nay ilcoin đang vượt qua khả_năng của btc và sở_hữu các giải_pháp như hệ_thống đô_đốc nodes hoặc công_nghệ mini block có đáng tin_cậy không là một đồng_tiền mã_hoá và là một đại_diện thay_thế cho để dùng giao_dịch thương_mại_điện_tử trên toàn_cầu với tốc_độ numbertoken giao_dịch mỗi giây tương_ứng với gb mỗi khối với thời_gian khai_thác khối trung_bình từ numbertoken numbertoken phút blockchain sử_dụng thuật_toán sha numbertoken và bằng_chứng công_việc proof of work cùng với numbertoken loại hợp_đồng thông_minh smart contract nó không phụ_thuộc vào hệ_thống ngân_hàng và có giá_trị độc_lập riêng lưu_giữ dài_hạn có_thể khiến cho tài_sản của bạn gia_tăng theo cấp_số nhân khi mà lượng nhu_cầu sử_dụng ngày_càng tăng cao khối khai_thác ban_đầu là numbertoken ilc numbertoken khối và cứ khoảng numbertoken năm tương_ứng với numbertoken numbertoken khối giảm đi numbertoken nữa cho đến khi đi tiệm_cận về numbertoken không và tổng cung của nó là numbertoken numbertoken tỷ đủ để toàn thế_giới vận_hành và đủ để tạo ra độ khan_hiếm hiện_tại đã khai_thác được numbertoken numbertoken numbertoken numbertoken ilc là tiền_tệ mã_hoá được sinh ra từ blockchain nó là một ứng_dụng đầu_tiên buộc phải có để cho blockchain vận_hành mà không cần một công_ty tập_đoàn cơ_quan hay chính_phủ nào can_thiệp được có các loại ví để lưu_trữ như ví web ví trên andriod ví apple osx qt ví mac os ví windows qt ví lạnh lần đầu_tiên được giao_dịch là vào ngày numbertoken numbertoken numbertoken hiện_nay ilcoin được niêm_yết trên hơn numbertoken sàn giao_dịch thuộc top numbertoken của thế_giới và hơn numbertoken trang thống_kê tiền_tệ mã_hoá như coinmarketcap com p s nếu bạn muốn tài_sản của mình có_thể gia_tăng một_cách đột_biến thì ilcoin là sự lựa_chọn hàng_đầu vì ilcoin là một trong những đồng_tiền kỹ_thuật_số tiên_tiến bậc nhất hiện_nay và có tiềm_năng gia_tăng giá_trị hàng trăm lần trong thời_gian tới nếu không muốn t